In [56]:
# check xgboost version
import xgboost
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
print(xgboost.__version__)

1.6.1


In [57]:
df = pd.read_csv('data/only2_big_with_sf.csv')
# df = df.transpose()
df.set_index('cell_line', inplace=True)
# df = df.transpose()
df.head()

,7105,8813,57147,55732,2268,3075,2519,2729,4800,90529,...,8336,84311,126661,79447,283521,29122,9997,388963,6905,SF
cell_line,,,,,,,,,,,,,,,,,,,,,
NCIH1568-2,5.626439,6.855242,2.185867,4.079805,0.084064,0.176323,6.824513,4.956521,3.766595,2.298658,...,0.622930,5.373996,1.956057,5.482203,0.422233,0.176323,0.918386,1.778209,4.050502,0.607940
SCC9-2,3.964399,5.718088,2.260026,3.451541,0.084064,0.782409,6.039796,5.778734,4.903038,2.711495,...,1.536053,5.042207,2.344828,3.937344,0.000000,0.042644,2.475085,1.176323,4.174726,0.573035
HCC1428-2,4.002703,5.870365,3.621759,3.861955,0.014355,0.485427,4.614121,5.286881,4.429616,3.442280,...,3.033863,4.539159,1.970854,5.895303,0.622930,0.014355,0.918386,2.244887,5.555816,0.450188
U2OS-2,5.098453,6.543496,2.117695,4.115200,0.014355,0.432959,5.684258,4.847997,5.108524,2.189034,...,0.176323,5.028569,2.967169,4.995937,0.111031,0.042644,2.010780,1.925999,4.635754,0.526517
LU99-2,3.665620,6.716717,2.100978,4.176323,0.000000,1.263034,6.124948,4.395063,5.070389,2.669027,...,1.321928,6.613679,1.438293,4.349082,0.000000,0.111031,1.111031,1.632268,5.197315,0.625161


In [58]:
# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.9

X = df.drop(columns = ['SF']).copy()
y = df['SF']

In [59]:
# In the first step we will split the data in training and remaining dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.9, random_state=42)

# Now since we want the valid and test size to be equal (10% each of overall data).
# we have to define valid_size=0.5 (that is 50% of remaining data)

print(X_train.shape), print(y_train.shape)
print(X_test.shape), print(y_test.shape)

(472, 15139)
(472,)
(53, 15139)
(53,)


(None, None)

In [60]:
# define model
model = XGBRegressor(tree_method='gpu_hist')

In [ ]:
%%time
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [ ]:
from numpy import absolute

# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [ ]:
# make a prediction
model = XGBRegressor()
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(X_test.shape)

In [ ]:
print(yhat)
print(y_test.values)

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
# print(y_test.tolist())
# print(y_pred_list)
plt.plot(y_test.tolist(), yhat, 'o')

a, b = np.polyfit(y_test.tolist(), yhat, 1)
plt.plot(y_test, a*y_test+b, color='steelblue', linestyle='--', linewidth=2)
slope, intercept, r_value, p_value, std_err = stats.linregress(y_test, yhat)
print("Slope:", slope)
print("Intercept:", intercept)
print("R-Squared:", r_value**2)
print("P-Value:", p_value)
print("Standard Error:", std_err)
plt.xlabel('Test Labels')
plt.ylabel('Predictions')
plt.title('Test Labels vs. Predictions')
plt.show()